In [ ]:
with 
base_total as
(
	select
		count(column_gold) as column_gold_t,
        count(column_ks) as column_ks_t,
        count(column_cpi) as column_cpi_t
	from third
),
base_merge as
(
	select
		a.date_input,
        a.column_gold,
        a.column_ks,
        a.column_cpi,
        a.gold_diff as gold_diff,
        a.ks_diff as ks_diff,
        a.cpi_diff as cpi_dff,
			count(case when a.gold_diff > 0 and a.ks_diff > 0 then 1 end) over (partition by date_input) as gold_ks_plus,
            count(case when a.gold_diff > 0 and a.cpi_diff > 0 then 1 end) over (partition by date_input) as gold_cpi_plus,
            count(case when a.ks_diff > 0 and a.cpi_diff > 0 then 1 end) over (partition by date_input) as ks_cpi_plus,
			count(case when a.gold_diff > 0 and a.ks_diff > 0 and a.cpi_diff > 0 then 1 end) over (partition by date_input) as all_plus,
            count(case when a.gold_diff < 0 and a.ks_diff < 0 then 1 end) over (partition by date_input) as gold_ks_minus,
            count(case when a.gold_diff < 0 and a.cpi_diff < 0 then 1 end) over (partition by date_input) as gold_cpi_minus,
            count(case when a.ks_diff < 0 and a.cpi_diff < 0 then 1 end) over (partition by date_input) as ks_cpi_minus,
            count(case when a.gold_diff < 0 and a.ks_diff < 0 and a.cpi_diff < 0 then 1 end) over (partition by date_input) as all_minus
		from
        (
			select
            date_input,
            column_gold,
            column_ks,
            column_cpi,
            round(lead(column_gold,1,column_gold) over(order by date_input) - column_gold,2) as gold_diff,
			round(lead(column_ks,1,column_ks) over(order by date_input)- column_ks,2) as ks_diff,
            round(lead(column_cpi,1,column_cpi) over(order by date_input)- column_cpi,2) as cpi_diff
		from third 
        ) a left join base_total b on 1=1
        group by a.date_input, a.gold_diff, a.ks_diff, a.cpi_diff, column_gold, column_ks, column_cpi
        order by a.date_input
) 
select
	*
from base_merge
;